In [2]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import lightgbm as lgb
from sklearn.model_selection import *
from sklearn.metrics import mean_squared_error, make_scorer
from scipy.stats import mode, skew, kurtosis, entropy
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")

transact_cols = [f for f in train.columns if f not in ["ID", "target"]]
y = np.log1p(train["target"]).values

In [4]:
nonzero_mean = y.mean()

nonzero_mean

14.490239292748585

In [5]:
test["target"] = nonzero_mean

In [163]:
cols_orig = ['f190486d6', '58e2e02e6', 'eeb9cd3aa', '9fd594eec', '6eef030c1', '15ace8c9f', 
        'fb0f5dbfe', '58e056e12', '20aa07010', '024c577b9', 'd6bb78916', 'b43a7cfd5', 
        '58232a6fb', '1702b5bf0', '324921c7b', '62e59a501', '2ec5b290f', '241f0f867', 
        'fb49e4212', '66ace2992', 'f74e8f13d', '5c6487af1', '963a49cdc', '26fc93eb7', 
        '1931ccfdd', '703885424', '70feb1494', '491b9ee45', '23310aa6f', 'e176a204a', 
        '6619d81fc', '1db387535', 
        'fc99f9426', '91f701ba2', '0572565c2', '190db8488', 'adb64ff71', 'c47340d97', 'c5a231d81', '0ff32eb98'
       ]

In [130]:
most_relevant = ['c5a231d81', 'c47340d97', '66ace2992', '2288333b4', '20aa07010', 'b43a7cfd5', '58e056e12', '6eef030c1', '1702b5bf0', '342e7eb03', 'ced6a7e91', 'fb387ea33', '491b9ee45', '58232a6fb', 'e222309b0', 'edc84139a', '9306da53f', 'fb0f5dbfe', '9f7b782ac', '61c1b7eb6', '64e38e7a2', 'd6bb78916', '50e4f96cf', '2ec5b290f', '26fc93eb7', '6619d81fc', '45f6d00da', '241f0f867', '70feb1494', 'f514fdb2e', '29ab304b9', 'ed8ff54b5', '2cb73ede7', '8a088af55', '1931ccfdd', 'd428161d9', 'b6fa5a5fd', '26ab20ff9', 'f1e0ada11', '72d34a148', '009319104', 'bee629024', 'd83a2b684', 'c1b9f4e76', 'e078302ef', '2eeadde2b', 'df838756c', '17b81a716', '58ed8fb53', '0c8063d63', 'f74e8f13d', '2ad744c57', '212efda42', 'd4e8dd865', 'bc70cbc26', '949ed0965', '324921c7b', '62e59a501', 'a8b721722', 'fb5e1b2b7', '20551fa5b', '83c3779bf', 'a4511cb0b', '402b0d650', '6cf7866c1', '6786ea46d', '8f3740670', 'ba4ceabc5', 'f39074b55', 'b98f3e0d7', '0d51722ca', '1d9078f84', '2fc60d4d9', '963a49cdc', '092271eb3', 'ed8951a75', '20604ed8f', '9df4daa99', '04b88be38', '371da7669', 'fec5eaf1a', 'fb49e4212', '0ff32eb98', '24b2da056', 'c7525612c', '1977eaf08', '350473311', '8337d1adc', '2b54cddfd', '467aa29ce', '542f770e5', '2488e17f5', '22d7ad48d', '0c9462c08', '467bee277', '0572565c2', '8e8736fc8', '703885424', 'e8f065c9d', '87ffda550', '0186620d7', 'bb1113dbb', 'b791ce9aa', 'f58fb412c', 'a75d400b8', 'fc99f9426', '1c71183bb', 'c928b4b74', '899dbe405', '68153d35e', '36a9a8479', '3f6246360', '5e645a169', 'f6eba969e', '73687e512', 'a19b05919', '64e483341', 'a6e871369', '4fa02e1a8', '37f57824c', '1db387535', 'adb64ff71', '190db8488', '29c059dd2', '453128993', '13ee58af1', '7121c40ee', '79cc300c7', '5c6487af1', '8479174c2', '44d5b820f', '580f5ff06', 'f1851d155', '68a945b18', '0e3ef9e8f', '7e814a30d', '94072d7a3', 'a93118262', 'fe33df1c4', 'aa31dd768', '861076e21', '5f341a818', '9280f3d04', '18c35d2ea']

In [131]:
other_cols = [x for x in most_relevant if x not in cols]

In [157]:
other_cols[0]

'2288333b4'

In [226]:
cols = cols_orig

In [227]:
# from: https://www.kaggle.com/dfrumkin/a-simple-way-to-use-giba-s-features-v2
def _get_leak(df, cols, lag=0):
    d1 = df[cols[:-lag-2]].apply(tuple, axis=1).to_frame().rename(columns={0: 'key'})
    d2 = df[cols[lag+2:]].apply(tuple, axis=1).to_frame().rename(columns={0: 'key'})
    d2['pred'] = df[cols[lag]]
    #d2 = d2[d2.pred != 0] ### to make output consistent with Hasan's function
    d3 = d2[~d2.duplicated(['key'], keep=False)]
    return d1.merge(d3, how='left', on='key').pred.fillna(0)

In [228]:
def compiled_leak_result():
    
    max_nlags = len(cols) - 2
    train_leak = train[["ID", "target"] + cols]
    train_leak["compiled_leak"] = 0
    train_leak["nonzero_mean"] = train[transact_cols].apply(
        lambda x: np.expm1(np.log1p(x[x!=0]).mean()), axis=1
    )
    
    scores = []
    leaky_value_counts = []
    leaky_value_corrects = []
    leaky_cols = []
    
    for i in range(max_nlags):
        c = "leaked_target_"+str(i)
        
        print('Processing lag', i)
        train_leak[c] = _get_leak(train_leak, cols, i)
        
        leaky_cols.append(c)
        train_leak = train.join(
            train_leak.set_index("ID")[leaky_cols+["compiled_leak", "nonzero_mean"]], 
            on="ID", how="left"
        )[["ID", "target"] + cols + leaky_cols+["compiled_leak", "nonzero_mean"]]
        zeroleak = train_leak["compiled_leak"]==0
        train_leak.loc[zeroleak, "compiled_leak"] = train_leak.loc[zeroleak, c]
        leaky_value_counts.append(sum(train_leak["compiled_leak"] > 0))
        _correct_counts = sum(train_leak["compiled_leak"]==train_leak["target"])
        leaky_value_corrects.append(_correct_counts/leaky_value_counts[-1])
        print("Leak values found in train", leaky_value_counts[-1])
        print(
            "% of correct leaks values in train ", 
            leaky_value_corrects[-1]
        )
        tmp = train_leak.copy()
        tmp.loc[zeroleak, "compiled_leak"] = tmp.loc[zeroleak, "nonzero_mean"]
        scores.append(np.sqrt(mean_squared_error(y, np.log1p(tmp["compiled_leak"]).fillna(nonzero_mean))))
        print(
            'Score (filled with nonzero mean)', 
            scores[-1]
        )
    result = dict(
        score=scores, 
        leaky_count=leaky_value_counts,
        leaky_correct=leaky_value_corrects,
    )
    return train_leak, result

In [229]:
train_leak, result = compiled_leak_result()

Processing lag 0
Leak values found in train 1351
% of correct leaks values in train  0.9955588452997779
Score (filled with nonzero mean) 1.5138333391635188
Processing lag 1
Leak values found in train 1947
% of correct leaks values in train  0.9964047252182845
Score (filled with nonzero mean) 1.2922048129527162
Processing lag 2
Leak values found in train 2340
% of correct leaks values in train  0.9935897435897436
Score (filled with nonzero mean) 1.1732829046778304
Processing lag 3
Leak values found in train 2586
% of correct leaks values in train  0.9930394431554525
Score (filled with nonzero mean) 1.084326373672634
Processing lag 4
Leak values found in train 2754
% of correct leaks values in train  0.9934640522875817
Score (filled with nonzero mean) 1.0327870440015579
Processing lag 5
Leak values found in train 2899
% of correct leaks values in train  0.9931010693342532
Score (filled with nonzero mean) 0.9940324299498775
Processing lag 6
Leak values found in train 3014
% of correct lea

In [230]:
result = pd.DataFrame.from_dict(result, orient='columns')
result.T

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
score,1.513833,1.292205,1.173283,1.084326,1.032787,0.994032,0.947572,0.906143,0.882938,0.864543,...,0.737684,0.737191,0.739431,0.738832,0.739338,0.740242,0.739891,0.740415,0.743261,0.742745
leaky_count,1351.000000,1947.000000,2340.000000,2586.000000,2754.000000,2899.000000,3014.000000,3110.000000,3188.000000,3237.000000,...,3633.000000,3637.000000,3644.000000,3647.000000,3650.000000,3660.000000,3662.000000,3663.000000,3666.000000,3666.000000
leaky_correct,0.995559,0.996405,0.993590,0.993039,0.993464,0.993101,0.992369,0.991961,0.991844,0.991659,...,0.985412,0.984603,0.983260,0.982451,0.981644,0.979508,0.979246,0.978979,0.978178,0.978178


In [231]:
result.to_csv('train_leaky_stat.csv', index=False)

In [232]:
train_leak.head()

,ID,target,f190486d6,58e2e02e6,eeb9cd3aa,9fd594eec,6eef030c1,15ace8c9f,fb0f5dbfe,58e056e12,...,leaked_target_30,leaked_target_31,leaked_target_32,leaked_target_33,leaked_target_34,leaked_target_35,leaked_target_36,leaked_target_37,compiled_leak,nonzero_mean
0,000d6aaf2,38000000.0,1866666.66,12066666.66,700000.0,600000.0,900000.0,4100000.0,0.0,0.0,...,0.0,38000000.0,0.0,38000000.0,38000000.0,0.0,0.0,0.0,38000000.0,2.858973e+06
1,000fbd867,600000.0,0.00,2850000.00,2225000.0,1800000.0,800000.0,0.0,0.0,3300000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,600000.0,4.303261e+06
2,0027d6b71,10000000.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,6000000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.473861e+06
3,0028cbf45,2000000.0,2000000.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2000000.0,4.017810e+05
4,002a68644,14400000.0,0.00,0.00,0.0,0.0,37662000.0,0.0,4000000.0,6700000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.567102e+06


In [233]:
best_score = np.min(result['score'])
best_lag = np.argmin(result['score'])
print('best_score', best_score, '\nbest_lag', best_lag)

best_score 0.7371911838169722 
best_lag 29


In [234]:
def rewrite_compiled_leak(leak_df, lag):
    leak_df["compiled_leak"] = 0
    for i in range(lag):
        c = "leaked_target_"+str(i)
        zeroleak = leak_df["compiled_leak"]==0
        leak_df.loc[zeroleak, "compiled_leak"] = leak_df.loc[zeroleak, c]
    return leak_df

In [235]:
leaky_cols = [c for c in train_leak.columns if 'leaked_target_' in c]
train_leak = rewrite_compiled_leak(train_leak, best_lag)
train_leak[['ID']+leaky_cols+['compiled_leak']].head()

,ID,leaked_target_0,leaked_target_1,leaked_target_2,leaked_target_3,leaked_target_4,leaked_target_5,leaked_target_6,leaked_target_7,leaked_target_8,...,leaked_target_29,leaked_target_30,leaked_target_31,leaked_target_32,leaked_target_33,leaked_target_34,leaked_target_35,leaked_target_36,leaked_target_37,compiled_leak
0,000d6aaf2,38000000.0,38000000.0,38000000.0,0.0,38000000.0,0.0,38000000.0,0.0,0.0,...,38000000.0,0.0,38000000.0,0.0,38000000.0,38000000.0,0.0,0.0,0.0,38000000.0
1,000fbd867,600000.0,0.0,0.0,0.0,0.0,0.0,600000.0,0.0,600000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,600000.0
2,0027d6b71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0028cbf45,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2000000.0
4,002a68644,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [236]:
train_res = train_leak[leaky_cols+['compiled_leak']].replace(0.0, np.nan)
train_res.to_csv('train_leak.csv', index=False)

In [237]:
def compiled_leak_result_test(max_nlags):
    test_leak = test[["ID", "target"] + cols]
    test_leak["compiled_leak"] = 0
    test_leak["nonzero_mean"] = test[transact_cols].apply(
        lambda x: np.expm1(np.log1p(x[x!=0]).mean()), axis=1
    )
    
    scores = []
    leaky_value_counts = []
    # leaky_value_corrects = []
    leaky_cols = []
    
    for i in range(max_nlags):
        c = "leaked_target_"+str(i)
        
        print('Processing lag', i)
        test_leak[c] = _get_leak(test_leak, cols, i)
        
        leaky_cols.append(c)
        test_leak = test.join(
            test_leak.set_index("ID")[leaky_cols+["compiled_leak", "nonzero_mean"]], 
            on="ID", how="left"
        )[["ID", "target"] + cols + leaky_cols+["compiled_leak", "nonzero_mean"]]
        zeroleak = test_leak["compiled_leak"]==0
        test_leak.loc[zeroleak, "compiled_leak"] = test_leak.loc[zeroleak, c]
        leaky_value_counts.append(sum(test_leak["compiled_leak"] > 0))
        #_correct_counts = sum(train_leak["compiled_leak"]==train_leak["target"])
        #leaky_value_corrects.append(_correct_counts/leaky_value_counts[-1])
        print("Leak values found in test", leaky_value_counts[-1])
        #print(
        #    "% of correct leaks values in train ", 
        #    leaky_value_corrects[-1]
        #)
        #tmp = train_leak.copy()
        #tmp.loc[zeroleak, "compiled_leak"] = tmp.loc[zeroleak, "nonzero_mean"]
        #scores.append(np.sqrt(mean_squared_error(y, np.log1p(tmp["compiled_leak"]).fillna(14.49))))
        #print(
        #    'Score (filled with nonzero mean)', 
        #    scores[-1]
        #)
    result = dict(
        # score=scores, 
        leaky_count=leaky_value_counts,
        # leaky_correct=leaky_value_corrects,
    )
    return test_leak, result

In [238]:
test_leak, test_result = compiled_leak_result_test(max_nlags=38)

Processing lag 0
Leak values found in test 2963
Processing lag 1
Leak values found in test 4215
Processing lag 2
Leak values found in test 4960
Processing lag 3
Leak values found in test 5503
Processing lag 4
Leak values found in test 5917
Processing lag 5
Leak values found in test 6208
Processing lag 6
Leak values found in test 6426
Processing lag 7
Leak values found in test 6583
Processing lag 8
Leak values found in test 6742
Processing lag 9
Leak values found in test 6872
Processing lag 10
Leak values found in test 6983
Processing lag 11
Leak values found in test 7080
Processing lag 12
Leak values found in test 7159
Processing lag 13
Leak values found in test 7243
Processing lag 14
Leak values found in test 7303
Processing lag 15
Leak values found in test 7366
Processing lag 16
Leak values found in test 7420
Processing lag 17
Leak values found in test 7462
Processing lag 18
Leak values found in test 7499
Processing lag 19
Leak values found in test 7540
Processing lag 20
Leak values 

In [239]:
test_result = pd.DataFrame.from_dict(test_result, orient='columns')
test_result.T

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
leaky_count,2963,4215,4960,5503,5917,6208,6426,6583,6742,6872,...,7835,7887,7949,8025,8117,8277,8478,8685,9012,9501


In [240]:
test_result.to_csv('test_leaky_stat.csv', index=False)

In [241]:
test_leak = rewrite_compiled_leak(test_leak, best_lag)
test_leak[['ID']+leaky_cols+['compiled_leak']].head()

,ID,leaked_target_0,leaked_target_1,leaked_target_2,leaked_target_3,leaked_target_4,leaked_target_5,leaked_target_6,leaked_target_7,leaked_target_8,...,leaked_target_29,leaked_target_30,leaked_target_31,leaked_target_32,leaked_target_33,leaked_target_34,leaked_target_35,leaked_target_36,leaked_target_37,compiled_leak
0,000137c73,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,00021489f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0004d7953,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,00056a333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,00056d8eb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [242]:
test_res = test_leak[leaky_cols+['compiled_leak']].replace(0.0, np.nan)
test_res.to_csv('test_leak.csv', index=False)

In [249]:
useful_info = test_leak[["ID", "compiled_leak"]]

In [250]:
useful_info.to_csv('leaked.csv', index=False)

In [43]:
#test_leak.loc[test_leak["compiled_leak"]==0, "compiled_leak"] = test_leak.loc[test_leak["compiled_leak"]==0, "nonzero_mean"]

In [44]:
#submission
sub = test[["ID"]]
sub["target"] = test_leak["compiled_leak"]
sub.to_csv(f"baseline_sub_lag_{best_lag}.csv", index=False)
print(f"baseline_sub_lag_{best_lag}.csv saved")

baseline_sub_lag_29.csv saved


In [45]:
sub.head()

,ID,target
0,000137c73,2.209645e+06
1,00021489f,1.099644e+06
2,0004d7953,1.276252e+06
3,00056a333,7.871320e+06
4,00056d8eb,2.868883e+06
